# PlasticParcels Example
## The pathways and fate of Italian coastal plastic pollution
In this example, we will use `PlasticParcels` to run a basic simulation of microplastic pollution along the Italian coastline.

In [ ]:
# Library imports
from datetime import datetime, timedelta

# Parcels and PlasticParcels imports
from parcels import ParticleFile                                                                                # type: ignore
import PlasticParcels as PP

# Load the model settings
settings = PP.utils.load_settings('PlasticParcels/Italy_simulation_settings.json')

# Create the simulation settings
settings['simulation'] = {
    'start_date': datetime.strptime('2019-01-10-00:00:00', '%Y-%m-%d-%H:%M:%S'), # Start date of simulation
    'runtime': timedelta(days=90),             # Runtime of simulation
    'dt_write': timedelta(hours=12),           # Timestep of output
    'dt_timestep': timedelta(minutes=20),      # Timestep of advection
    }

# Overwrite some settings if desired
settings['mode'] = '2D'
settings['biofouling_f'] = False
settings['stokes_f'] = True
settings['wind_f'] = True

# Create a fieldset with required data
fieldset = PP.constructors.create_fieldset(settings)

# Create the particle release settings
settings['release'] = {
    'wind_coefficient' : 0.01,  # Percentage of wind to apply to particles
    'plastic_diameter' : 0.001, # Plastic particle diameter (m)
    'plastic_density' : 1030.,  # Plastic particle density (kg/m^3)
    'initialisation_type': 'coastal',
    'country': 'Italy',   
}

# Create the particleset
pset = PP.constructors.create_particleset_from_map(fieldset, settings)

# Create the applicable kernels to the plastic particles
kernels = PP.constructors.create_kernel(fieldset)

# Define the runtime, the timestepping, and the output frequency of the simulation from the settings
runtime = settings['simulation']['runtime']
dt_timestep = settings['simulation']['dt_timestep']
dt_write = settings['simulation']['dt_write']

# Create the particle file where output will be stored
pfile = ParticleFile('Italian_coastal_release.zarr', pset, outputdt=dt_write)
pfile.add_metadata('PlasticParcels_settings', str(settings))

# Execute the simulation
pset.execute(kernels, runtime=runtime, dt=dt_timestep, output_file=pfile)


In [ ]:
settings['ocean']['indices'] = {'lon':range(3300, 4000), 'lat':range(1850, 2400)}#, 'depth':range(0,2)}

In [ ]:
# Create the fieldset
fieldset = PP.constructors.create_fieldset(settings)
fieldset.computeTimeChunk()

In [ ]:
fig = plt.figure(figsize=(10, 5.5))
ax=plt.subplot(111, projection=ccrs.PlateCarree())
cbar = ax.pcolormesh(fieldset.U.grid.lon, fieldset.U.grid.lat, fieldset.U.data[0,0,:,:], transform=ccrs.PlateCarree(), cmap=plt.cm.bwr, vmin=-0.75, vmax=0.75)
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title('Surface current zonal velocity (m/s)')
plt.colorbar(cbar,shrink=0.75, pad=0.075)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5.5))
ax=plt.subplot(111, projection=ccrs.PlateCarree())
cbar = ax.pcolormesh(fieldset.Wind_U.grid.lon, fieldset.Wind_U.grid.lat, fieldset.Wind_U.data[0,:,:], transform=ccrs.PlateCarree(), cmap=plt.cm.RdGy, vmin=-20, vmax=20)
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title('Zonal wind velocity (m/s)')
plt.colorbar(cbar,shrink=0.75, pad=0.075)
plt.show()

In [1]:
# Library imports
from datetime import datetime, timedelta

# Parcels and PlasticParcels imports
from parcels import ParticleFile                                                                                # type: ignore
from PlasticParcels import constructors
import PlasticParcels as PP
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import tqdm

In [2]:
plt.close('all')

In [4]:
## Create the animations

# Load the model settings
settings = PP.utils.load_settings('PlasticParcels/lorenz_settings.json')
settings['ocean']['indices'] = {'lon':range(3300, 4000), 'lat':range(1850, 2400), 'depth':range(0,2)}

# Create the simulation settings

settings['simulation'] = {
    'start_date': datetime.strptime('2020-01-01-00:00:00', '%Y-%m-%d-%H:%M:%S'), # Start date of simulation
    'runtime': timedelta(days=90),             # Runtime of simulation
    'dt_write': timedelta(hours=12),           # Timestep of output
    'dt_timestep': timedelta(minutes=20),      # Timestep of advection
    }
settings['wind_f'] = True


for time_id in tqdm.tqdm(range(120)):
    settings['simulation']['start_date'] = settings['simulation']['start_date'] + timedelta(days=time_id)
    # Create the fieldset
    fieldset = PP.constructors.create_fieldset(settings)
    fieldset.computeTimeChunk()


    fig = plt.figure(figsize=(10, 5.5))
    ax=plt.subplot(111, projection=ccrs.PlateCarree())
    cbar = ax.pcolormesh(fieldset.U.grid.lon, fieldset.U.grid.lat, fieldset.U.data[0,0,:,:], transform=ccrs.PlateCarree(), cmap=plt.cm.bwr, vmin=-0.75, vmax=0.75)
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    plt.title('Surface current zonal velocity (m/s)')
    plt.colorbar(cbar,shrink=0.75, pad=0.075)
    plt.savefig('fig/currents_'+str(time_id)+'.png', dpi=150, bbox_inches='tight')
    plt.close('all')

    fig = plt.figure(figsize=(10, 5.5))
    ax=plt.subplot(111, projection=ccrs.PlateCarree())
    cbar = ax.pcolormesh(fieldset.Wind_U.grid.lon, fieldset.Wind_U.grid.lat, fieldset.Wind_U.data[0,:,:], transform=ccrs.PlateCarree(), cmap=plt.cm.RdGy, vmin=-20, vmax=20)
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    plt.title('Zonal wind velocity (m/s)')
    plt.colorbar(cbar,shrink=0.75, pad=0.075)
    plt.savefig('fig/wind_'+str(time_id)+'.png', dpi=150, bbox_inches='tight')
    plt.close('all')


  0%|          | 0/120 [00:00<?, ?it/s]/nethome/denes001/.conda/envs/parcels-v3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1785: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  result = super().pcolormesh(*args, **kwargs)
/nethome/denes001/.conda/envs/parcels-v3/lib/python3.10/site-packages/cartopy/mpl/geoaxes.py:1785: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  result = super().pcolormesh(*args, **kwargs)
  1%|          | 1/120 [00:10<19:52, 10.02s

OSError: FieldSet files not found for variable Wind_U: []

In [ ]:
datetime.strptime('2019-01-01-00:00:00', '%Y-%m-%d-%H:%M:%S')+timedelta(days=1)

In [ ]:
# Create the fieldset
fieldset = PP.constructors.create_fieldset(settings)

In [ ]:
# Create the particleset
pset = PP.constructors.create_particleset_from_map(fieldset, settings)

In [ ]:
# Create the applicable kernels to the plastic particles
kernels = PP.constructors.create_kernel(fieldset)

In [ ]:
# Define the runtime, the timestepping, and the output frequency of the simulation from the settings
runtime = settings['simulation']['runtime']
dt_timestep = settings['simulation']['dt_timestep']
dt_write = settings['simulation']['dt_write']



In [ ]:
# Create the particle file where output will be stored
pfile = ParticleFile('example_Italy_coast.zarr', pset, outputdt=dt_write)
pfile.add_metadata('PlasticParcels_settings', str(settings))

In [ ]:
# Execute the simulation
pset.execute(kernels, runtime=runtime, dt=dt_timestep, output_file=pfile)

In [ ]:
import xarray as xr
ds = xr.open_zarr('example_Italy_coast.zarr')

In [ ]:
ds.plastic_amount

In [ ]:
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines(zorder=20)
ax.plot(ds['lon'].T, ds['lat'].T, transform=ccrs.PlateCarree())

plt.show()

In [ ]:
import numpy as np
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines(zorder=20)
minval = np.min(ds['plastic_amount'].T[0,:])
maxval = np.max(ds['plastic_amount'].T[0,:])
for i in range(len(ds['plastic_amount'].T[0,:])):
    ax.plot(ds['lon'].T[:,i], ds['lat'].T[:,i], c=plt.cm.jet(ds['plastic_amount'].T[0,i]), transform=ccrs.PlateCarree())
plt.show()

In [ ]:
ds['plastic_amount'].T[0,:].values

In [ ]:
ds['plastic_amount'].T[0].values.shape

In [ ]:
kernels